## Zadanie 2
### Hubert Kłosowski 242424
### Kamil Małecki 242464

### Potrzebne importy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from scipy.stats import iqr, zscore

## Wczytanie danych

In [ ]:
def get_data():
    os.chdir('..')
    os.chdir('Zadanie1\\czesc4')
    data = [pd.read_csv(filename) for filename in os.listdir()]
    os.chdir('..')
    os.chdir('..')
    os.chdir('Zadanie2')
    return data

dataframes = get_data()

## <center>Część 1</center>
### Wykonaj wykresy pudełkowe (ang.box plot), które pozwolą na graficzną identyfikację wartości odstających poprzez przedstawienie kwartyli i wartości skrajnych w danych dotyczących:

### 1.1 liczby nowych zachorowań

In [ ]:
sns.boxplot(data=dataframes[1], y='daily_confirmed', linewidth=2)

### 1.2 liczby nowych śmierci

In [ ]:
sns.boxplot(data=dataframes[2], y='daily_deceased', linewidth=2)

### 1.3 liczby nowych szczepień

In [ ]:
sns.boxplot(data=dataframes[3], y='daily_persons_vaccinated', linewidth=2)

### 1.4 populacji

In [ ]:
sns.boxplot(data=dataframes[0], y='population', linewidth=2)

### 1.5 PKB

In [ ]:
sns.boxplot(data=dataframes[0], y='gdp_usd', linewidth=2, log_scale=10)

### 1.6
### średnia długość życia w krajach

In [ ]:
sns.boxplot(data=dataframes[0], y='life_expectancy', linewidth=2)

### PKB na osobę

In [ ]:
sns.boxplot(data=dataframes[0], y='gdp_per_capita_usd', linewidth=2)

### cena benzyny w USA i UK

In [ ]:
sns.boxplot(data=dataframes[-1], y='Gasoline Prices $/litre', x='CCA3', linewidth=2)

### liczba przestępstw z podziałem na jego rodzaj w Południowej Australii

In [ ]:
sns.boxplot(data=dataframes[-3], x='Offence count', y='Offence Type', linewidth=2)

## <center>Część 2</center>
<ol>
    <li style='font-size: 20px'>Metoda Z-Score</li>
    <p style='font-size: 20px'>Metoda statystyczna, która mierzy, jak wiele odchyleń standardowych wartość jest oddalona od średniej.<br>Wartości z wysokim Z-Score (np. powyżej 2 lub 3) mogą być uznane za wartości odstające, a następnie usunięte. </p>
    <br>
    <li style='font-size: 20px'>Metoda IQR (ang. InterQuartile Range)</li>
    <p style='font-size: 20px'>Technika oparta na kwartylach, w której oblicza się zakres międzykwartylowy (Q3 - Q1) i wartości odstające są określane jako wartości leżące poza granicami 1,5 IQR poniżej Q1 lub powyżej Q3. </p>
</ol>

In [ ]:
def del_outliers(series, method='zscore'):
    if method == 'zscore':
        x = zscore(series)
        for index in range(len(x)):
            if -2 <= x[index] >= 2:
                series.pop(index)
    elif method == 'iqr':
        q1, q3, c = series.quantile(0.25), series.quantile(0.75), iqr(series) * 1.5
        for index, v in series.items():
            if q1 - c <= v >= q3 + c:
                series.pop(index)
    else:
        raise ValueError('Nie ma takich rzeczy')
    return series

to_delete = [dataframes[0]['population'], dataframes[0]['gdp_per_capita_usd'], dataframes[1]['daily_confirmed'], dataframes[2]['daily_deceased'], dataframes[3]['daily_persons_vaccinated']]
after_zscore = [del_outliers(el) for i, el in enumerate(to_delete) if i % 2 == 0]
after_iqr = [del_outliers(el, method='iqr') for i, el in enumerate(to_delete) if i % 2 != 0]

## <center>Część 3</center>
### Wykonaj następujące obliczenia:

### 3.1. średnia liczba nowych zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,

### 3.2. średnia zmiana liczby zachorowań, śmierci i szczepień dla co najmniej 10 wybranych krajów w wybranym miesiącu,